In [ ]:
import pandas
import numpy
from scipy.io import loadmat#读取mat文件

# 多分类问题的逻辑回归
先把数据读取出来看看是啥  
有5000个样本，每个特征有400给，就是20*20的像素，里面是手写的数字

In [ ]:
def getData(url):
    data=loadmat(url)
    row=data['X'].shape[0]
    x=numpy.insert(data['X'],0,numpy.ones(row),axis=1)
    y=numpy.array(data['y'])
    theta=numpy.zeros((x.shape[1],1))
    return x,y,theta


然后定义sigmoid函数、代价函数和梯度下降的函数  
sigmoid函数公式如下  
 $$
 sigmoid=\frac{1}{1+e^{-z}}
 $$
代价函数公式如下
$$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}+\frac{\lambda }{2m}\sum\limits_{j=1}^{n}{\theta _{j}^{2}}$$
梯度下降公式如下(这是争对一个特征j的下降公式，实际使用中通过矩阵可以一次计算出所有特征下降公式)
$$\frac{\partial J\left( \theta  \right)}{\partial {{\theta }_{j}}}=\left( \frac{1}{m}\sum\limits_{i=1}^{m}{\left( {{h}_{\theta }}\left( {{x}^{\left( i \right)}} \right)-{{y}^{\left( i \right)}} \right)} \right)x_{_{j}}^{(i)}+\frac{\lambda }{m}{{\theta }_{j}}\text{ }\text{             for  j}\ge \text{1}$$

In [ ]:
def sigmoid(z):
    return 1/(1+numpy.exp(-z))

def cost(theta,x,y,Lambda=1):
    row=x.shape[0]
    hx=x@theta
    return (1/row)*(((-y.T)@numpy.log(sigmoid(hx)))-(((1-y).T)@numpy.log(1-sigmoid(hx))))+(Lambda/(2*row))*(theta.T@theta)

def gradientDescent(theta,x,y,Lambda=1):
    row=x.shape[0]
    return ((1/row)*x.T@(sigmoid(x@theta)-y)+(Lambda/row)*theta)

有了上面的的三个公式，就可以求二分类问题了，现在将多分类问题转换成多个二分类问题就可以了
对于识别0-9，可以变成对0与其他分类，1与其他分类等等，产生10个theta
训练完成后，对于新的测试数据，只需要用10个测试数据计算，大于0.5的就是该数了

In [ ]:
import copy
import scipy.optimize as opt
import matplotlib.pyplot as plt
def one_to_all():
    x,y,theta=getData('ex3data1.mat')
    thetas=[]
    for j in range(1,11):   
        nums=copy.deepcopy(y)
        thetaCopy=copy.deepcopy(theta)
        nums[y==j]=1
        nums[y!=j]=0
        costs=[]
        for i in range(2000):
            costs.append(cost(thetaCopy,x,nums)[0][0])
            thetaCopy=thetaCopy-gradientDescent(thetaCopy,x,nums)
        thetas.append(thetaCopy)
        print(j,end='')
        # plt.plot(costs)
        # plt.show()
    numpy.save('nums',thetas)
one_to_all()

得到了0-9学习后的特征，将数据拿去验算，看看正确率有多少

In [5]:
from random import randrange

#太离谱了，每个下降1000，正确率0.931。下降2000后，正确率0.936
#答案是0.9446，还差了点
def checkRight():
    thetas=numpy.array(numpy.load('nums.npy'))[:,:,0]
    x,y,theta=getData('ex3data1.mat')
    ans=numpy.argmax(x@thetas.T,axis=1)[:,numpy.newaxis]
    rigth=0
    for i in range(len(ans)):
        if(ans[i]+1==y[i]):
            rigth+=1
return rigth/len(ans)
    #testgithub
checkRight()